In [1]:
import sys
sys.path.append('..')

In [2]:
import keras_multilabel_embedding as kml
import tensorflow as tf

## Multi-label embedding (fixed number of labels)

In [3]:
x_ids = [[1, 2, 4], [0, 1, 2], [2, 1, 4], [3, 2, 1]]
x_ids = tf.constant(x_ids)

layer1a = kml.MultiLabelEmbedding(
    vocab_size=500000, embed_size=300, random_state=42)

%timeit y1a = layer1a(x_ids)

2022-02-28 16:48:47.987081: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


910 µs ± 88.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%time 
for _ in range(100):
    y1a = layer1a(x_ids)
print(y1a.shape)

(4, 300)
CPU times: user 83.4 ms, sys: 5.55 ms, total: 88.9 ms
Wall time: 116 ms


## Multi-label embedding (variable number of labels)

In [5]:
x_ids = [[1, 2, 4], [0, 1, 2], [2, 1], [3]]

layer1b = kml.MultiLabelEmbedding2(
    vocab_size=500000, embed_size=300, random_state=42)

%timeit y1b = layer1b(x_ids)

2.06 ms ± 448 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%time 
for _ in range(100):
    y1b = layer1b(x_ids)
print(y1b.shape)

(4, 300)
CPU times: user 212 ms, sys: 10.4 ms, total: 222 ms
Wall time: 244 ms


## Simple dense embedding

In [7]:
# x_ids = [[1, 2, 4], [0, 1, 2], [2, 1, 4], [3, 2, 1]]
x_mask = [[0., 1, 1, 0, 1], [1, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 0]]
x_mask = [ex + [0]*(500000-5) for ex in x_mask]
x_mask = tf.constant(x_mask)

layer2 = tf.keras.layers.Dense(300, use_bias=False)
tf.keras.utils.set_random_seed(42)
layer2.build(input_shape=(None, 500000))
layer2.set_weights(layer1a.get_weights()) 

%timeit y2 = layer2(x_mask)

70.2 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%time
for _ in range(10):
    y2 = layer2(x_mask)
print(y2.shape)

(4, 300)
CPU times: user 1.91 s, sys: 25.3 ms, total: 1.93 s
Wall time: 660 ms
